# Week 4: Machine learning 1

Machine learning is a technique for learning patterns in data that enable computers to make decisions and predictions. It's probably one of the hottest skills to master as a scientist or engineer in research or industry today. This week, we'll get an overview of what machine learning is, what it can be used for and what its limits are. Without worrying too much about what goes on behind the scenes, we will play around with a few classifiers in Python and test model performance using cross validation. The exercises today cover:

* Feature representation
* Model fitting
* Model evaluation
* Prediction results

[**Feedback**](http://ulfaslak.com/vent)

## Exercises

We want to predict whether a character is a hero or a villain from information that we can extract from their markup. This is a large problem that includes some data wrangling, model fitting and a bit of evaluation. Therefore the problem is split into parts.

### Part 1: Feature representation
In it's raw format, the data cannot be given to a machine learning algorithm. What we must do is extract features from the data and put them into a structured format. This is the same as what we did when we looked at a dog (the data) and extracted into a matrix whether it was fluffy, sad looking, etc. (the features). The feature we will extract here is **team alliances**.

We can represent the team alliances of each character as a row in a matrix where each column corresponds to a particular team. That should look something like this (numbers are made up):

<img src="http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_boa.png" width="400"/>

**Note**: The following exercises relies on the dataset you produced in Ex. 3.1.2 (character markup stored on your computer). If you didn't manage to produce the dataset [use mine](http://github.com/ulfaslak/caobd_s19/tree/master/data) and go back and complete it at a later time so that you get the most out of this session.

> **Ex.4.1.1**: Write a function called `get_alliances` that takes the name of a character and returns a list of teams that the character is allied with. Print the alliances of Iron Man.

*Hint: There is a place in the character markup where you'll find the team affiliations and you want to write some code that can extract that. The affiliations are not always listed the same way so you should probably try to eyeball some of the documents to get a feel of how the formatting can look.*

In [1]:
import re, os

def get_alliances(char, faction=None):
    """Return list of alliances for Marvel character."""
    
    if faction is None:
        for faction in ["heroes", "ambiguous", "villains"]:
            faction_chars = [c[:-4] for c in os.listdir("/Users/alderik/Documents/School/College/DIS/bigdata/caobd_f19/data/%s" % faction)]
            if char in faction_chars:
                break
    
    # Load character markup
    with open("/Users/alderik/Documents/School/College/DIS/bigdata/caobd_f19/data/%s/%s.txt" % (faction, char)) as fp:
        markup = fp.read()

    # Get alliance field
    alliances_field = re.findall(r"alliances[\w\W]+?\|.+=", markup)
    if alliances_field == []:
        return []

    # Extract teams from alliance field
    return [t[2:-1] for t in re.findall(r"\[\[.+?[\]\|]", alliances_field[0][10:])]

In [2]:
get_alliances("Iron Man")

['Avengers (comics)',
 'United States Department of Defense',
 'Force Works',
 'The New Avengers (comics)',
 'Guardians of the Galaxy (2008 team)',
 'Illuminati (comics)',
 'The Mighty Avengers',
 'S.H.I.E.L.D.',
 'Stark Industries',
 'Stark Resilient',
 'Thunderbolts (comics)']

> **Ex.4.1.2**: Produce a sorted list called `all_teams` that contains all teams in the entire Marvel universe. Print the first 10.

In [3]:
def get_universe():
    directory = "/Users/alderik/Documents/School/College/DIS/bigdata/caobd_f19/data/"
    marvel_universe = []
    for faction in ["heroes", "ambiguous", "villains"]:
        faction_chars = [c[:-4] for c in os.listdir("/Users/alderik/Documents/School/College/DIS/bigdata/caobd_f19/data/%s" % faction)]
        for i in faction_chars:
            marvel_universe.append(i)
            
    marvel_universe.sort()
    return marvel_universe

In [4]:
def get_teams():
    """Returns list of all teams"""
    all_teams = []
    
    chars = get_universe()
    for i in chars:
        teams = get_alliances(i)
        while len(teams)>0:
            if teams[0] not in all_teams:
                all_teams.append(teams[0])
                teams.remove(teams[0])
            else:
                teams.remove(teams[0])
    
    all_teams.sort()
    return all_teams

In [35]:
ans = get_teams()
ans

['A-Force',
 'A-Next',
 'A.R.M.O.R.',
 'AXIS (comics)',
 'Acolytes (comics)',
 'Acts of Vengeance',
 'Advanced Idea Mechanics',
 'Advanced Idea Mechanics#Avengers Idea Mechanics',
 'Age of Apocalypse',
 'Agency X',
 'Agent X (Marvel Comics)',
 'Agents of Atlas',
 'Agents of Fortune (comics)',
 'Alchemax',
 'Alias (comics)',
 'All-New X-Factor',
 'All-New X-Men',
 'All-New, All-Different Avengers',
 'All-Winners Squad',
 'Alliance of Evil',
 'Alliance of Evil (comics)',
 'Alpha Flight',
 'Alpha Squadron (comics)',
 'Amazons',
 'America Redeemers',
 'Anachronauts (comics)',
 'Ani-Men',
 'Annihilation (comics)',
 'Annihilators (Marvel Comics)',
 'Apocalypse (comics)',
 'Apocalypse: The Twelve',
 'Argonauts',
 'Art Thibert',
 'Arthur Adams (comics)',
 'Asgard (comics)',
 'Askani',
 'Astronaut',
 'Atlantis (Marvel Comics)',
 'Attuma',
 'Avengers (comics)',
 'Avengers 1959',
 'Avengers A.I.',
 'Avengers Academy',
 'Avengers West Coast',
 'Avengers: The Initiative',
 'Avengers: The Initiative

> **Ex.4.1.3**: Write a function that takes the name of a character and returns a vector representation of the team alliances for that character. The length of the returned list will be `len(all_teams)` and have 0s in the places corresponding to teams that the character is not on, and 1s in the places corresponding to teams the character is on. Print the sum of the list returned when the input to the function is Iron man.

In [6]:
def get_vector(char):
    faction = None
    char_alliances = get_alliances(char, faction)
    all_teams = get_teams()
    
    alliance_vector = []
    for i in all_teams:
        count = 0
        for j in char_alliances:
            if i==j:
                count=1
                alliance_vector.append(count)
        alliance_vector.append(count)
        
    return alliance_vector

In [7]:
sum(get_vector('Iron Man'))

22

> **Ex. 4.1.4**: Create the team alliance matrix for the data. This is your feature matrix for the classification problem you will solve later in this exercise set. Therefore, you should also—in a seperate *target* array—store whether characters are heros (denote by 1) or villains (denote by 0). For now, skip ambiguous characters, but write your code in such a way that it won't be too hard to redo this for ambiguous characters. Skip characters that have no team alliances. Print the shapes of your matrix and target array.

In [8]:
import pandas as pd
def get_vector_quick(char, all_teams):
    faction = None
    char_alliances = get_alliances(char, faction)
    alliance_vector = []
    for i in all_teams:
        count = 0
        for j in char_alliances:
            if i==j:
                count=1
                alliance_vector.append(count)
            else:
                alliance_vector.append(count)
        
    return alliance_vector

def get_vector_quick(char, all_teams):
    faction = None
    char_alliances = get_alliances(char, faction)
    alliance_vector = []
    for team in all_teams:
        if team in char_alliances:
            alliance_vector.append(1)
        else:
            alliance_vector.append(0)
    return alliance_vector

def get_matrix():
    all_chars = get_universe()
    all_teams = get_teams()
    char_vecs = []
    
    for i in all_chars:
        char_vecs.append(get_vector_quick(i,all_teams))
        
    return char_vecs

In [11]:
char_alliance_vec = get_matrix()
all_teams = get_teams()
all_chars = get_universe()
df = pd.DataFrame(char_alliance_vec, columns=all_teams, index=all_chars) 
df

,A-Force,A-Next,A.R.M.O.R.,AXIS (comics),Acolytes (comics),Acts of Vengeance,Advanced Idea Mechanics,Advanced Idea Mechanics#Avengers Idea Mechanics,Age of Apocalypse,Agency X,...,Yancy Street Gang,Young Allies,Young Allies (Marvel Comics),Young Allies (Marvel Comics)#Counter-Earth,Young Allies (Marvel Comics)#Heroes Reborn,Young Allies (Marvel Comics)#Heroic Age,Young Avengers,Young Masters,Young X-Men,Zodiac (comics)
Abigail Brand,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Abner Jenkins,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Abomination (comics),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Abraham Cornelius,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Absalom (comics),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Absorbing Man,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Abyss (comics),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Achilles (Pantheon),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acrobat (comics),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adam II,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
def get_faction(char):
    faction = None
    ans = ''
    if faction is None:
        for faction in ["heroes", "ambiguous", "villains"]:
            faction_chars = [c[:-4] for c in os.listdir("/Users/alderik/Documents/School/College/DIS/bigdata/caobd_f19/data/%s" % faction)]
            if char in faction_chars:
                ans = faction
                break
    return ans
    
def hero_vil_ambig_matrix():
    all_chars = get_universe()
    faction_id = []
    for i in all_chars:
        faction = get_faction(i)
        if faction =='villains':
            faction_id.append(1)
        elif faction =='heroes':
            faction_id.append(0)
        else:
            faction_id.append(2)
    return faction_id

In [19]:
target_array = hero_vil_ambig_matrix()
df['faction'] = target_array #adds column of factions

In [34]:
df_final = df[df.faction != 2]

### Part 2: Model fitting

> **Ex. 4.2.1**: Train a classifier on all of your data and test its accuracy.

>* If your team alliance matrix is `X_ta` and your target array is `y_ta` you can do this by instantiating a model like:
>
        from sklearn.naive_bayes import BernoulliNB
        model = BernoulliNB()
        model.fit(X_ta, y_ta)  # <--- This is the training/fitting/learning step
        
> The `BernoulliNB` is a version of the Naive Bayes classifier which associates certain features with labels and asks what the probability of a label for a data point is given its features. You are free to use any other classifier if you want. Popular ones are trees, random forests, support vector machines, feed forward neural networks, logistic regression, and the list goes on. With `sklearn`, they are just as easy to employ as the `BernoulliNB` classifier.


>1. Test the accuracy of your model. You can use the `.predict` method on the `model` object to get predictions for a matrix of data points. Report the accuracy of your model on the same data that you trained the model on, alongside the baseline accuracy of a "dumb" model that only guesses for the majority class.

>2. Report the precision, recall and F1 scores, with respect to the minority class (heroes). `sklearn` has implementations that you can use if you are short for time. Extra credit for doing it using only basic linear algebra operations with `numpy`, though.

### Part 3: Model evaluation

> **Ex. 4.3.1**: Investigate how well your model generalizes. You may have noticed that the performance seemed a little too good to be true in Ex 4.2.1.
1. Why did you get such a high accuracy in the previous exercise?
2. Split your data into a test and training set of equal size. Train the model only on the training set and report its accuracy and F1 scores (for both classes) on both the training and test sets.
3. Comment on the difference you observe.

> *Hint: Watch out for unbalanced class proportions! You may want to randomly reorder the rows of your datapoints and target labels so your training and test sets have the same amount of heroes and villains.*

> **Ex. 4.3.2**: Implement cross validation. The performance of a classifier is strongly dependent on the amount of data it is trained on. In Ex. 4.3.1 you train it on only half of the data and test it on the other half. If you rerun that code multiple times, with random 50/50 partitions, you are gonna see a lot of uncertainty in performance. Cross validation solves this problem by training on a larger subset of the data and testing on a smaller one, and taking the average performance over K-folds of this process.
1. Implement cross validation over $K=10$ folds. For each fold you must record the training and test accuracies. In the end, visualize the distributions of test- and training accuracy as histograms in the same plot. It's important that you comment on the result.

### Part 4: Predicting good vs. evil

>**Ex. 4.4.1**: Let's put our classifier to use!
* Retrain your model on all of your data.
* Create a team alliance representation of the ambiguous characters
* Use the model the estimate the probability that each character is a villain (let's call this *villainness*). You can use the `.predict_proba` method on the model to get probability estimates rather than class assignments.
* **Visualize the "heroness" distribution for all ambiguous characters**. Comment on the result.